# Python json

json是JavaScript Object Notation的简写，是一种轻量级的数据交换格式。  
JSON建构于两种结构：

* “名称/值”对的集合。
* 值的有序列表（An ordered list of values）。

Python内置了json模块用于将python对象转换为json字符串和将json字符串转换为python对象。 

## python对象转换为json字符串

使用json模块提供的dump或dumps函数将python对象转换为json字符串。  

语法：json.dump(obj, fp, *, skipkeys=False, ensure_ascii=True, check_circular=True, allow_nan=True, cls=None, indent=None, separators=None, default=None, sort_keys=False, **kw)。  

dumps和dump的参数类似，只不过dumps是将对象转换为json字符串，而dump则将转换后的json字符串直接写入传给的文件中。  
转换规则：

| Python | JSON |
|:-------|:-----|
| dict | object |
| list, tuple | array |
| str | string |
| int, float, int- & float-derived Enums | number |
| True | true |
| False | false |
| None | null |

In [2]:
import json
json.dumps(['foo', {'bar': ('baz', None, 1.0, 2)}])

'["foo", {"bar": ["baz", null, 1.0, 2]}]'

### skipkeys
默认为False，如果字典的key不是str, int, float, bool, None之一，将报TypeError。如果skipkeys=True，则不会报错，而是跳过解释该键值对。

不加skipkeys参数：

In [4]:
class A():
    pass

try:
    json.dumps({1: "a", A(): "b"})
except TypeError as e:
    print(e)

keys must be a string


添加skipkeys参数:

In [5]:
json.dumps({1: "a", A(): "b"}, skipkeys=True)

'{"1": "a"}'

### allow_nan
默认为True，能够将float('nan')转换为NaN，float('inf')转换为Infinity，float('-inf')转换为-Infinity。

In [6]:
json.dumps({"a": float('nan'), "b": float('inf'), "c": float("-inf")})

'{"a": NaN, "b": Infinity, "c": -Infinity}'

如果设置allow_nan为False，则遇到上面三种情况会抛出ValueError。毕竟这三种情况并不是太标准的json值。

In [7]:
try:
    json.dumps({"a": float('nan'), "b": float('inf'), "c": float("-inf")}, allow_nan=False)
except ValueError as e:
    print(e)

Out of range float values are not JSON compliant


### indent
设置缩进，一般只为了测试时好看，传送数据在乎的是尽量减少大小。

In [12]:
print(json.dumps(['foo', {'bar': ('baz', None, 1.0, 2)}], indent=4))

[
    "foo",
    {
        "bar": [
            "baz",
            null,
            1.0,
            2
        ]
    }
]


### separators
separators的值为一个两个元素的元组：(item_separator, key_separator)，分别是项和键值之间的分隔符，默认为(",", ": ")，注意:后面有一个空格，如果为了尽量缩小字符串的大小，可以设置为(",", ":")。

In [8]:
print(json.dumps(['foo', {'bar': ('baz', None, 1.0, 2)}]))

["foo", {"bar": ["baz", null, 1.0, 2]}]


In [9]:
print(json.dumps(['foo', {'bar': ('baz', None, 1.0, 2)}], separators=(",", ":")))

["foo",{"bar":["baz",null,1.0,2]}]


### sort_keys
是否按字典的key排序，默认是False。

默认是无序的：

In [10]:
print(json.dumps({"c": 20, "a": 50, "b": 70}))

{"c": 20, "a": 50, "b": 70}


设置为有序：

In [11]:
print(json.dumps({"c": 20, "a": 50, "b": 70},sort_keys=True))

{"a": 50, "b": 70, "c": 20}


## json字符串转换为python对象

使用json模块提供的load或loads函数将json字符串转换为python对象。  

语法：json.load(fp, *, cls=None, object_hook=None, parse_float=None, parse_int=None, parse_constant=None, object_pairs_hook=None, **kw)。  

loads和load的参数类似，只不过loads是将json字符串转换为对象，而load则直接读取传给的文件中的内容转换为对象。  
转换规则：

| JSON | Python |
|:-----|:-------|
|object | dict |
|array | list |
|string | str |
|number (int) |	int |
|number (real) | float |
|true |	True |
|false | False |
|null |	None |

In [12]:
text = '["foo", {"bar": ["baz", null, 1.0, 2]}]'
lst = json.loads(text)
print(type(lst))

<class 'list'>


In [13]:
print(lst[1]["bar"])

['baz', None, 1.0, 2]


### parse_float
接受一个函数，对json中的浮点值进行加工。默认为float(num_str)，将json字符串中的浮点数字符串强制转换为浮点数。所以这个参数是作为是以原始浮点数字符串为输入，进行特殊处理后返回处理结果。

In [15]:
text = '{"a": 1.2}'
d = json.loads(text, parse_float=lambda x: float(x) + 10)
d

{'a': 11.2}

### parse_int
类似parse_float

In [18]:
text = '{"a": 5}'
d = json.loads(text, parse_int=lambda x: int(x) + 10)
d

{'a': 15}

## 个性化转换

###  转换对象
默认json模块是不知道如何转换对象到json字符串的，但你可以告诉json如何转换。有两种方法。

第一种方法：

In [26]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

def object2dict(obj):
    d = {}
    if obj.__class__ == Person:
        d['name'] = obj.name
        d['age'] = obj.age
    return d

json.dumps([1.2, "hello", {"aa": "bbb"}, Person('Alan', 32)], default=object2dict)

'[1.2, "hello", {"aa": "bbb"}, 100]'

1. 给`json.dumps`或`json.dump`方法的default参数赋值一个函数
2. 该函数用于处理非一般的对象，返回一般对象（数值、字符串、字典、列表、true、false、null）。

第二种方法：

1. 编写自己的Encoder继承json.JSONEncoder。
2. 覆盖default方法。
3. 创建自己的Encoder对象，调用encode方法将对象转换为json字符串。

In [41]:
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        d = {}
        if obj.__class__ == Person:
            d['name'] = obj.name
            d['age'] = obj.age
        return d

MyEncoder().encode([1.2, "hello", {"aa": "bbb"}, Person('Alan', 32)])

'[1.2, "hello", {"aa": "bbb"}, {"name": "Alan", "age": 32}]'



### 转换字符串
默认json模块是将json对象转换python字典的，但你可以告诉json将json对象转换为python的特定对象。有两种方法。

第一种方法：

In [35]:
def obj2complex(d):
    if '__complex__' in d:
        return complex(d['real'], d['imag'])
    return d # 如果只是普通的字典，一定要写上这局，因为是对所有字典操作

json.loads('[1.2, {"aa": "bb"}, {"real": 5, "imag": 4, "__complex__": true}]', object_hook=obj2complex)

[1.2, {'aa': 'bb'}, (5+4j)]

1. 给`json.load`或`json.loads`方法的object_hook参数赋值一个函数
2. 该函数用于处理json字符串中的对象进行定制处理

第二种方法：

1. 编写自己的Decoder继承json.JSONDecoder。
2. 将转换函数移到自己的Decoder中，在初始化的时候，指定object_hook
3. 创建自己的Decoder对象，调用decode方法将json对象转换为python对象。

In [44]:
class MyDecoder(json.JSONDecoder):
    def __init__(self):
        json.JSONDecoder.__init__(self,object_hook=self.obj2complex)
    def obj2complex(self, d):
        if '__complex__' in d:
            return complex(d['real'], d['imag'])
        return d
    
MyDecoder().decode('[1.2, {"aa": "bb"}, {"real": 5, "imag": 4, "__complex__": true}]')

[1.2, {'aa': 'bb'}, (5+4j)]